In [147]:
from bokeh.plotting import figure, output_file, show
import pandas as pd
import os
import numpy as np
from bokeh.io import output_notebook
from IPython.display import display, HTML


from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.tools import CustomJSHover
from bokeh.palettes import Dark2_5 as palette
import itertools #for cycling through colors 
from bokeh.models import Legend

In [148]:
output_notebook()

Loading BokehJS ...

In [166]:
df = pd.read_csv("/var/folders/rt/610_wrfj70q8221h55lj8rvr009rz8/T/tmp_3sukv0y")

In [167]:
df = df.drop(df.index[18])
df = df.drop(df.index[18])
display(df)

,StartTime,ActualDuration,Labels,NumThreads,ActualQPS,p50,p90,p99,cpu_mili_avg_telemetry_mixer,cpu_mili_max_telemetry_mixer,mem_MB_max_telemetry_mixer,cpu_mili_avg_fortioserver_deployment_proxy,cpu_mili_max_fortioserver_deployment_proxy,mem_MB_max_fortioserver_deployment_proxy,cpu_mili_avg_ingressgateway_proxy,cpu_mili_max_ingressgateway_proxy,mem_MB_max_ingressgateway_proxy
0,2019-06-05T18:13:01.976651739Z,180,373f14ef_qps_1000_c_60_1024_base,60,1000,945,1448,2753,-,-,-,2,4,28,5,6,24
1,2019-06-05T18:10:01.244726937Z,180,373f14ef_qps_1000_c_60_1024_both,60,1000,6270,7847,8982,-,-,-,165,210,28,5,9,24
2,2019-06-05T18:07:00.492606919Z,180,373f14ef_qps_1000_c_60_1024_serveronly,60,1000,3652,4778,5969,-,-,-,117,135,27,4,6,24
3,2019-06-05T18:03:59.816182493Z,180,373f14ef_qps_1000_c_50_1024_base,50,1000,867,1248,2260,-,-,-,3,4,27,4,9,24
4,2019-06-05T18:00:59.156316612Z,180,373f14ef_qps_1000_c_50_1024_both,50,1000,5497,6820,7971,-,-,-,156,289,27,4,6,24
5,2019-06-05T17:57:58.474955516Z,180,373f14ef_qps_1000_c_50_1024_serveronly,50,1000,3172,4153,5315,-,-,-,115,164,27,4,8,24
6,2019-06-05T17:54:57.798853844Z,180,373f14ef_qps_1000_c_40_1024_base,40,1000,858,1198,2073,-,-,-,3,5,26,5,6,24
7,2019-06-05T17:51:57.202427134Z,180,373f14ef_qps_1000_c_40_1024_both,40,1000,4496,5583,6687,-,-,-,182,294,26,5,8,24
8,2019-06-05T17:48:56.514466777Z,180,373f14ef_qps_1000_c_40_1024_serveronly,40,1000,3112,3874,4817,-,-,-,125,174,26,4,5,24
9,2019-06-05T17:45:55.947264584Z,180,373f14ef_qps_1000_c_30_1024_base,30,1000,789,1046,1880,-,-,-,3,4,26,4,4,24


In [151]:
# format title 
percentile="p90"
qps=df.at[0, 'ActualQPS'] 
seconds=df.at[0, 'ActualDuration']

title="Istio {} Latency, {} QPS over {} seconds".format(percentile, qps, seconds)
print(title)

Istio p90 Latency, 1000 QPS over 180 seconds


In [152]:
# get series (y axis)
modes = ['base', 'both', 'serveronly'] 
series = {}
for m in modes:
    #print(m)
    rows = df[df.Labels.str.endswith(m)]
    #display(rows)
    vals = list(rows[percentile]) # get latency values for this mode - NOTE -reversing
    vals.reverse()
    series[m] = vals 

# get x axis (connections) 
c = list(rows.NumThreads)
c.sort()
print(c)

print(series)

[10, 20, 30, 40, 50, 60]
{'base': [742, 893, 1046, 1198, 1248, 1448], 'both': [2422, 3490, 4875, 5583, 6820, 7847], 'serveronly': [1293, 2220, 2943, 3874, 4153, 4778]}


In [163]:
# output to static HTML file
fn = "".join(title.split())
output_file(fn + ".html")

# create a new plot
p = figure(
   tools="pan,box_zoom,reset,save",
    title=title,
    plot_width=1000, plot_height=600,
   x_axis_label='connections', y_axis_label='P90 latency, micros'
)

# turn off minor ticks; increase font sizes
p.title.text_font_size = '22pt'
p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
p.xaxis.axis_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.xaxis.major_label_text_font_size = "13pt"
p.yaxis.major_label_text_font_size = "13pt"


# plot latency data for each mode 
colors = itertools.cycle(palette) 

for mode, latency in series.items(): 
    col = next(colors)
    p.line(c, latency, line_color=col)
    p.circle(c, latency, legend=mode, size=10, fill_color=col)

In [164]:
# add hover tools to show latency values per circle 
p.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
    ('latency (s)', '$data_y')
]))

In [165]:
show(p)

In [23]:
df["Labels"] = [ x.split('_', 6)[-1] for x in df['Labels']]
df["p90"] = df["p90"] / 1000.0  #convert latency from ms to seconds 

In [24]:
for label in df["Labels"].unique():
    ll = df[df["Labels"]==label]
    ll.sort_values(by=["NumThreads"])
    
    p = figure(
   tools="pan,box_zoom,reset,save",
        title="p90 Latency with increasing # connections",
   x_axis_label='connections', y_axis_label='P90 latency, seconds'
)
    
    p.circle(ll["NumThreads"], ll["p90"], name=label, legend=label)